# 95/20 Ranking

Estimate top-coded incomes from PUMs data. It sounds like we should pull with a PUMA instead of MET2013, and translate the geography in the analysis.

In [28]:
import pandas

ct_census = pandas.read_csv("too_big/ss14hct.csv") ## Grab for every topcoded state

pums = pandas.read_csv("too_big/usa_00002.csv")
acs_14 = pandas.read_csv("hh_14.csv",encoding = "ISO-8859-1")

In [30]:
census_vars = ["PUMA","ADJINC","SERIALNO","HINCP","WGTP"]

#bridgeport_pumas = ["00100", "00101", "00102", "00103", "00104", "00105"]

ct_census["bridgeport"] = ct_census["PUMA"].apply(lambda x: ((x>=100) & (x<=105)))

bridgeport_2014 = ct_census[ct_census["bridgeport"]==True][census_vars]
bridgeport_2014["HHINC"] = pandas.to_numeric(bridgeport_2014["HINCP"],errors="coerce")
bridgeport_2014["ADJHH"] = bridgeport_2014.apply(lambda x: (x["ADJINC"]/1000000)*x["HHINC"], axis=1)

In [31]:
def multiply(row,income_col,weight_col,tar):
    i = 0
    while i < row[weight_col]:
        new = {"PUMA":row["PUMA"],income_col:row[income_col]}
        tar.append(new)
        i+=1

def explode_weights(df,income_col, weight_col):
    ## Make a list of HHINCOME for calculating
    ## a quantile where each household's HHINCOME
    ## is repeated in the series n times, where
    ## n == weight_col
    tar = []
    df.apply(lambda x: multiply(x,income_col,weight_col,tar),axis=1)
    return tar

exploded_bridgeport = explode_weights(bridgeport_2014,"ADJHH","WGTP")

In [32]:
pandas.DataFrame(exploded_bridgeport)["ADJHH"].quantile(.95, interpolation="lower")

558969.9774999999

# Hallelujah!

This is the number Brookings had! But from talking to the Census it sounds like they were wrong to adjust income to constant dollars, so we won't do that for our analysis:

In [33]:
exploded_bridgeport = explode_weights(bridgeport_2014,"HHINC","WGTP")
pandas.DataFrame(exploded_bridgeport)["HHINC"].quantile(.95)

554300.0

# What's left:

#1 Write the functions to streamline it
#2 Gather the necessary 2016 data: ACS & PUMS
#3 Put them where they need to be & fill out the 2016 data set
#4 Do the analysis for 2007, too, and get the over-time chart (or maybe figure out when EPI measured it's recession numbers from?)

* identify all the topcoded entries -- get the right state PUMs files ugh such a pain

* find the corresponding PUMAs in MABLE/GEOCORR, and figure out what to do if any of them cross a CBSA border

* extract the correct data from the PUMs files... a lot of this might be manual

In [34]:
acs_16 = pandas.read_csv("hh_16.csv", encoding = "ISO-8859-1")
mable = pandas.read_csv("geocorr15.csv", encoding = "ISO-8859-1")[1:]

top_coded = acs_16[acs_16["HD01_VD06"].isnull()]
bottom_coded = acs_16[acs_16["HD01_VD02"].isnull()]

top_coded["States"] = top_coded["GEO.display-label"].apply(lambda x: x.split(',')[1].split()[0].split('-'))
#top_coded["States"] = top_coded["States"].drop_duplicates()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [35]:
mable.head()

,state,puma12,cbsa,stab,cbsaname15,PUMAname,pop10,afact
1,06,00101,41860,CA,"San Francisco-Oakland-Hayward, CA (Metro)",Alameda County (North)--Berkeley & Albany Cities,131119,1
2,06,00102,41860,CA,"San Francisco-Oakland-Hayward, CA (Metro)",Alameda County (Northwest)--Oakland (Northwest...,158091,1
3,06,00103,41860,CA,"San Francisco-Oakland-Hayward, CA (Metro)",Alameda County (Northeast)--Oakland (East) & P...,125229,1
4,06,00104,41860,CA,"San Francisco-Oakland-Hayward, CA (Metro)",Alameda County (North Central)--Oakland City (...,124608,1
5,06,00105,41860,CA,"San Francisco-Oakland-Hayward, CA (Metro)","Alameda County (West)--San Leandro, Alameda & ...",163534,1


In [36]:
mable["cbsa"] = mable["cbsa"].apply(lambda x : pandas.to_numeric(x,errors="coerce"))
all_pumas = []


def get_pumas(geo_id,geo_corr):
    cbsa = geo_corr[geo_corr["cbsa"] == geo_id] 
    if len(cbsa) == 0:
        print(geo_id)
    target = {}
    for index,row in cbsa.iterrows(): # do i need "index"?
        target[int(row["puma12"])] = float(row["afact"])
        all_pumas.append(int(row["puma12"]))
    return target

top_coded["PUMAS"] = top_coded["GEO.id2"].apply(lambda x: get_pumas(x,mable))

640354


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [37]:
top_coded.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,States,PUMAS
0,1600000US0640354,640354,"La Quinta city, California",27794.0,2167.0,55920,4923,90161,6006,146206,6942,NaN,NaN,[California],{}
109,310M300US14460,14460,"Boston-Cambridge-Newton, MA-NH Metro Area",29225.0,345.0,60545,343,98251,581,154820,789,NaN,NaN,"[MA, NH]","{400: 0.346, 501: 1.0, 502: 1.0, 503: 1.0, 504..."
110,310M300US14500,14500,"Boulder, CO Metro Area",28368.0,911.0,55378,1196,91056,1521,148663,2650,NaN,NaN,[CO],"{801: 0.211, 802: 0.704, 803: 1.0, 804: 0.361}"
120,310M300US14860,14860,"Bridgeport-Stamford-Norwalk, CT Metro Area",32385.0,674.0,66551,945,110368,1393,191602,2122,NaN,NaN,[CT],"{100: 1.0, 101: 1.0, 102: 1.0, 103: 1.0, 104: ..."
259,310M300US20780,20780,"Edwards, CO Micro Area",40478.0,2582.0,66183,4343,94777,5378,139984,3936,NaN,NaN,[CO],{400: 0.46}


In [40]:
import zipfile,requests,io,os

def fetch_census(x):
    zip_path = "https://www2.census.gov/programs-surveys/acs/data/pums/2016/1-Year/csv_h"+x+".zip"
    r = requests.get(zip_path, stream=True)
    zip_ref = zipfile.ZipFile(io.BytesIO(r.content))
    dir_path = "too_big/2016/csv_h"+x+"/"
    zip_ref.extractall(dir_path)
    os.remove("too_big/2016/csv_h"+x+"/ACS2016_PUMS_README.pdf")

def download_unzip_census(ss_s):
    ss_s.apply(lambda x: fetch_census(x.lower()))
    
#fetch_census("ct")

In [41]:
def make_path(state):
    return ("too_big/2016/csv_h"+state+"/ss16h"+state+".csv")
    
def get_census_pums(ss_s,model):
    tar = pandas.DataFrame(columns=model.columns)
    for index, s in ss_s.iteritems():
        file = make_path(s.lower())
        new = pandas.read_csv(file) ## Grab for every topcoded state
        tar = pandas.concat([tar,new])
    return tar

In [47]:
import pickle

def write_obj(obj,filename):
    with open(filename, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_obj(filename):
    with open(filename, 'rb') as handle:
        b = pickle.load(handle)
        return b
    


In [74]:
def pums_dataset():
    if (os.path.isfile("too_big/exploded_census.pickle")):
        return load_obj("too_big/exploded_census.pickle")
    else:
        s = pandas.Series(top_coded["States"].sum()).drop_duplicates().sort_values()
        s = s[s!="California"]
        if not (os.path.isfile("too_big/2016/csv_hco/ss16hco.csv")):
            download_unzip_census(s)
        census_pums = get_census_pums(s,ct_census)
        census_pums["relevant"] = census_pums["PUMA"].apply(lambda x: x in all_pumas)
        relevant_census = census_pums[census_pums["relevant"]]
        exploded_census = explode_weights(relevant_census,"HINCP","WGTP")
        write_obj(exploded_census,"too_big/exploded_census.pickle")
        return exploded_census

#
# I thiiink this makes sense but we'll see when i pull it and try it at home
#
exploded_census = pums_dataset()
exploded_census = pandas.DataFrame(exploded_census)
exploded_census["HHINC"] = pandas.to_numeric(exploded_census["HINCP"],errors="coerce")

In [ ]:
# make dict of FIPS: pandas.DataFrame(pums rows)
# explode weights FIRST

def get_data(puma,percent,census):
    all_entries = census[census["PUMA"]==puma]
    if percent < 1: 
        s = all_entries.sample(frac=percent,random_state=0)
        return(s)
    return(all_entries)
    
def make_fips_data_dict(df,census):
    if (os.path.isfile("too_big/fips_dict.pickle")):
        return load_obj("too_big/fips_dict.pickle")
    else:
        d = {}
        for index, row in df.iterrows():
            tar = pandas.DataFrame(columns=census.columns)
            fip = row["GEO.id2"]
            pumas = row["PUMAS"]
            for p in pumas:
                new = get_data(p,pumas[p],census)
                tar = pandas.concat([tar,new])
            d[fip] = tar
        write_obj(d,"too_big/fips_dict.pickle")
        return d
    
fips_data_dict = make_fips_data_dict(top_coded,exploded_census)

In [79]:
def get_quantile(fips,percent,d):
    data = d[fips]
    return(data["HHINC"].quantile(.95))

top_coded["95_Percentile"] = top_coded["GEO.id2"].apply(lambda x: get_quantile(x,.95,fips_data_dict))

KeyError: 'HHINC'

In [77]:
top_coded

,GEO.id,GEO.id2,GEO.display-label,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,States,PUMAS,95_Percentile
0,1600000US0640354,640354,"La Quinta city, California",27794.0,2167.0,55920,4923,90161,6006,146206,6942,NaN,NaN,[California],{},NaN
109,310M300US14460,14460,"Boston-Cambridge-Newton, MA-NH Metro Area",29225.0,345.0,60545,343,98251,581,154820,789,NaN,NaN,"[MA, NH]","{400: 0.346, 501: 1.0, 502: 1.0, 503: 1.0, 504...",240000.0
110,310M300US14500,14500,"Boulder, CO Metro Area",28368.0,911.0,55378,1196,91056,1521,148663,2650,NaN,NaN,[CO],"{801: 0.211, 802: 0.704, 803: 1.0, 804: 0.361}",219000.0
120,310M300US14860,14860,"Bridgeport-Stamford-Norwalk, CT Metro Area",32385.0,674.0,66551,945,110368,1393,191602,2122,NaN,NaN,[CT],"{100: 1.0, 101: 1.0, 102: 1.0, 103: 1.0, 104: ...",290000.0
259,310M300US20780,20780,"Edwards, CO Micro Area",40478.0,2582.0,66183,4343,94777,5378,139984,3936,NaN,NaN,[CO],{400: 0.46},199300.0
417,310M300US27220,27220,"Jackson, WY-ID Micro Area",32402.0,5774.0,59526,4904,85468,5229,129621,9133,NaN,NaN,"[WY, ID]","{1100: 0.078, 100: 0.196}",195000.0
506,310M300US31060,31060,"Los Alamos, NM Micro Area",47250.0,8691.0,88420,3310,123242,8281,177485,9948,NaN,NaN,[NM],{300: 0.143},191800.0
507,310M300US31080,31080,"Los Angeles-Long Beach-Anaheim, CA Metro Area",24426.0,149.0,48258,212,79542,294,130673,456,NaN,NaN,[CA],"{3701: 1.0, 3702: 1.0, 3703: 1.0, 3704: 1.0, 3...",250000.0
563,310M300US33260,33260,"Midland, TX Metro Area",31622.0,922.0,57037,2005,89892,3126,140136,4595,NaN,NaN,[TX],"{2800: 0.047, 3000: 1.0}",245000.0
601,310M300US34900,34900,"Napa, CA Metro Area",32283.0,1433.0,59536,1913,93005,2113,146169,3402,NaN,NaN,[CA],{5500: 1.0},236000.0


In [78]:
fips_data_dict[14460]["HINCP"].quantile(.95)

240000.0

In [67]:
ct_census_2016 = pandas.read_csv(make_path("ct"))
ct_census_2016["bridgeport"] = ct_census_2016["PUMA"].apply(lambda x: ((x>=100) & (x<=105)))
bridgeport_2016 = ct_census_2016[ct_census_2016["bridgeport"]==True][census_vars]
bridgeport_2016["HHINC"] = pandas.to_numeric(bridgeport_2016["HINCP"],errors="coerce")
exploded_bridgeport_2016 = explode_weights(bridgeport_2016,"HHINC","WGTP")
pandas.DataFrame(exploded_bridgeport_2016)["HHINC"].quantile(.95)

482000.0

In [68]:
exploded_bridgeport_2016

[{'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 {'HHINC': 157000.0, 'PUMA': 102.0},
 

# Qunitile Wages 2007-2016

Track either mean wages per quintile or quintile cut-offs over time, to show the stratification of the recession and the recovery. Probably want to do this for something like:

* CT (state)
* CT (urban areas)
* Bridgeport Metro Area
* New Haven Metro Area
* Bridgeport as city

OR do this or mean income urban vs non-urban w/in Connecticut?

So you need a call for each year -- and the url construction might change year to year, though I think it's mostly just 2016 that's special for some reason.

A bunch of maybe-useful api queries:

    * https://api.census.gov/data/2016/acs/acs1/variables.html
    * https://api.census.gov/data/2016/acs/acs1/examples.html
    * https://api.census.gov/data/2016/acs/acs1?get=B01001_001E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*
    * https://api.census.gov/data/2015/acs1/examples.html
    * https://api.census.gov/data/2016/
            

In [ ]:
import json,urllib,re,numpy,time,config

base = "https://api.census.gov/data/"

def construct_url(year,geo_type,geo_code):
    url = base+str(year)+"/acs/acs1?get="+config.fec_key+"&is_amended=False&per_page=100"
    return(url)

def lookup(cid, t=5):
    url = construct_url(cid)
    data = url_to_data(url)
    if t>0:
        time.sleep(t)
    if data is None:
        return None
    return data["results"]

## If the fn can't find a response at the url, it returns None
## That gets converted into an empty dataframe (not ideal), and
## and flagged for the second pass
def url_to_data(url):
    try:
        req = urllib.request.Request(url)
        req.add_header('User-agent', 'Mozilla 5.10')
        res = urllib.request.urlopen(req)
        data = json.load(res)
        return data
    except urllib.error.HTTPError as err:
        return None
    
lookup("C00406389")